In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# define training settings
NUM_EPOCHS = 150
BATCH_SIZE = 128

In [3]:
import torch

torch.cuda.set_device(2)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [4]:
from doren_bnn.mobilenet import MobileNet, NetType
from torchinfo import summary

NETTYPE = NetType.XNORPP_SCA
model = MobileNet(3, num_classes=10, nettype=NETTYPE).to(device)

summary(model, input_size=(BATCH_SIZE, 3, 32, 32))

Layer (type:depth-idx)                             Output Shape              Param #
MobileNet                                          [128, 10]                 --
├─Upsample: 1-1                                    [128, 3, 224, 224]        --
├─Sequential: 1-2                                  [128, 1024, 1, 1]         --
│    └─MobileNet_ConvBlock: 2-1                    [128, 32, 112, 112]       --
│    │    └─Sequential: 3-1                        [128, 32, 112, 112]       902
│    └─MobileNet_ConvDsBlock: 2-2                  [128, 64, 112, 112]       --
│    │    └─Sequential: 3-2                        [128, 64, 112, 112]       2,560
│    └─MobileNet_ConvDsBlock: 2-3                  [128, 128, 56, 56]        --
│    │    └─Sequential: 3-3                        [128, 128, 56, 56]        9,216
│    └─MobileNet_ConvDsBlock: 2-4                  [128, 128, 56, 56]        --
│    │    └─Sequential: 3-4                        [128, 128, 56, 56]        18,304
│    └─MobileNet_ConvDsB

In [5]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR

criterion = CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=5e-6)
# scheduler = CosineAnnealingWarmRestarts(optimizer, 25, eta_min=1e-4)
scheduler = CosineAnnealingLR(optimizer, NUM_EPOCHS)

In [6]:
# hyperparameters
ALPHA = 1e-2
LAMBDA = 1e-5

In [7]:
from doren_bnn.utils import Dataset, Experiment

EXPERIMENT_ID = f"mobilenet-xnorpp-sca-cifar10-full-{ALPHA}-{LAMBDA}"
print(EXPERIMENT_ID)
experiment = Experiment(EXPERIMENT_ID, Dataset.CIFAR10, BATCH_SIZE)

mobilenet-xnorpp-sca-full-0.01-1e-05
Files already downloaded and verified
Files already downloaded and verified


In [8]:
experiment.train(
    device,
    model,
    criterion,
    optimizer,
    scheduler,
    NUM_EPOCHS,
    resume=False,
    alpha=ALPHA,
    lamb=LAMBDA,
)

  0%|          | 0/150 [00:00<?, ?it/s]